# Student Management System
**Project by EduTech Solutions**

This notebook implements a **Student Management System** that allows users to manage student records with secure login functionality, CRUD operations, and file-based storage.
    The notebook is fully documented to provide a clear understanding of each step.

---

## Step 1: Project Structure

The project will store files in the same directory as the notebook:

- `users.txt` → Stores user credentials
- `students.txt` → Stores student records
- `activity_log.txt` → Logs all user activities

This structure ensures separation of data and logging for better maintainability.


In [1]:
# Step 2: Import Required Modules

# os → interact with the file system
# getpass → securely capture passwords
# datetime → timestamp for logging user activities
import os
import getpass
import datetime


## Step 3: File Paths

Define paths for storing users, students, and activity logs.


In [2]:
# File paths
USER_FILE = "users.txt"
STUDENT_FILE = "students.txt"
LOG_FILE = "activity_log.txt"


## Step 4: Authentication System

The authentication system ensures secure access. Functions include:

1. `log_activity` → logs user actions with timestamp
2. `load_users` → loads user credentials from file
3. `save_user` → saves new user credentials
4. `authenticate` → checks login details against stored records


In [3]:
def log_activity(username, action):
    """Log user actions with timestamp"""
    with open(LOG_FILE, "a") as log:
        log.write(f"{datetime.datetime.now()} - {username} - {action}\n")

def load_users():
    """Load all user credentials from file"""
    users = {}
    if os.path.exists(USER_FILE):
        with open(USER_FILE, "r") as f:
            for line in f:
                if "," in line:
                    uname, pwd = line.strip().split(",", 1)
                    users[uname] = pwd
    return users    

def save_user(username, password):
    """Save new user to the file"""
    with open(USER_FILE, "a") as f:
        f.write(f"{username},{password}\n")        

def authenticate():
    """Authenticate user login"""
    users = load_users()
    username = input("Enter Username: ")
    password = getpass.getpass("Enter Password: ")
    if username in users and users[username] == password:
        log_activity(username, "Logged in")
        print("\n✅ Login Successful!\n")
        return username
    else:
        print("\n❌ Invalid credentials!\n")
        return None


## Step 5: Student Data Management (CRUD)

These functions manage student records stored in `students.txt`:

- **Add Student** → Adds a new record
- **View Students** → Lists all student records
- **Search Student** → Finds a student by roll number
- **Update Student** → Modifies an existing record
- **Delete Student** → Removes a student record


In [4]:
def add_student(username):
    """Add a new student to the records"""
    roll = input("Enter Roll Number: ")
    name = input("Enter Name: ")
    grade = input("Enter Grade: ")
    with open(STUDENT_FILE, "a") as f:
        f.write(f"{roll},{name},{grade}\n")
    log_activity(username, f"Added student {roll}")
    print("✅ Student added successfully!")    

def view_students():
    """View all student records"""
    if not os.path.exists(STUDENT_FILE):
        print("No records found.")
        return
    with open(STUDENT_FILE, "r") as f:
        print("\n--- Student Records ---")
        for line in f:
            roll, name, grade = line.strip().split(",")
            print(f"Roll: {roll} | Name: {name} | Grade: {grade}")

def search_student():
    """Search for a student by Roll Number"""
    roll_no = input("Enter Roll Number to search: ")
    found = False            
    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE, "r") as f:
            for line in f:
                parts = line.strip().split(",")
                if len(parts) != 3:
                    continue
                roll, name, grade = parts
                if roll == roll_no:
                    print(f"✅ Found: Roll: {roll} | Name: {name} | Grade: {grade}")
                    found = True
                    break 

def update_student(username):
    """Update details of an existing student"""
    roll_no = input("Enter Roll Number to update: ")
    students = [] 
    updated = False
    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE, "r") as f:
            students = f.readlines()
        with open(STUDENT_FILE, "w") as f:
            for line in students:
                roll, name, grade = line.strip().split(",")
                if roll == roll_no:
                    print(f"Current Data → Name: {name}, Grade: {grade}")
                    new_name = input("Enter new Name: ")
                    new_grade = input("Enter new Grade: ")
                    f.write(f"{roll},{new_name},{new_grade}\n")
                    updated = True
                    log_activity(username, f"Updated student {roll}")
                else:
                    f.write(line)
                        
    if updated:
        print("✅ Student record updated.")
    else:
        print("❌ Student not found.")    

def delete_student(username):
    """Delete a student record"""
    roll_no = input("Enter Roll Number to delete: ")
    students = [] 
    deleted = False
    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE, "r") as f:
            students = f.readlines()
        with open(STUDENT_FILE, "w") as f:
            for line in students:
                parts = line.strip().split(",")
                if len(parts) != 3:
                    f.write(line)  # keep invalid lines unchanged
                    continue
                roll, name, grade = parts
                if roll == roll_no:
                    deleted = True
                    log_activity(username, f"Deleted student {roll}")
                else:
                    f.write(line)    
    if deleted:
        print("✅ Student record deleted.")
    else:
        print("❌ Student not found.")     


## Step 6: Report Generation

Generate a basic summary report of student records, showing total students and grade distribution.


In [5]:
def generate_report():
    """Generate a summary report of students"""
    if not os.path.exists(STUDENT_FILE):
        print("No records available.")
        return
    
    total = 0
    grades = {}
    with open(STUDENT_FILE, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) != 3:
                continue
            roll, name, grade = parts
            total += 1
            grades[grade] = grades.get(grade, 0) + 1

    print("\n📊 Student Report")
    print(f"Total Students: {total}")
    for g, count in grades.items():
        print(f"Grade {g}: {count} student(s)")


## Step 7: Main Program

This is the entry point. It ensures an admin exists, handles user authentication, and provides a menu-driven interface for all functionalities.


In [6]:
def main():
    print("===== Student Management System =====")
    # Ensure admin user exists
    if not os.path.exists(USER_FILE):
        print("No users found. Create an Admin account.")
        uname = input("Set Admin Username: ")
        pwd = getpass.getpass("Set Admin Password: ")
        save_user(uname, pwd)
        print("✅ Admin created! Please restart the program.")
        return

    username = None
    while not username:
        username = authenticate()
    
    while True:
        print("\n--- Main Menu ---")
        print("1. Add Student")
        print("2. View All Students")
        print("3. Search Student")
        print("4. Update Student")
        print("5. Delete Student")
        print("6. Generate Report")
        print("7. Logout & Exit")
        
        choice = input("Enter choice: ") 

        if choice == "1":
            add_student(username)
        elif choice == "2":
            view_students()
        elif choice == "3":
            search_student()
        elif choice == "4":
            update_student(username)
        elif choice == "5":
            delete_student(username)
        elif choice == "6":
            generate_report()
        elif choice == "7":
            log_activity(username, "Logged out")
            print("👋 Goodbye!")
            break
        else:
            print("❌ Invalid choice! Please try again.")

# Run main program
if __name__ == "__main__":         
    main()


===== Student Management System =====


Enter Username:  admin
Enter Password:  ········



✅ Login Successful!


--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  1
Enter Roll Number:  101
Enter Name:  Jhone
Enter Grade:  A


✅ Student added successfully!

--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


...


--- Student Records ---
Roll: 10 | Name: John | Grade: 10
Roll: 12 | Name: Abuki | Grade: 9
Roll: 12 | Name: Barkola | Grade: 9
Roll: 100 | Name: Dawa | Grade: A
Roll: 103 | Name: Bruk | Grade: A
Roll: 103 | Name: Agmas | Grade: C
Roll: 101 | Name: Adane | Grade: B
Roll: 103 | Name: Bereket | Grade: C

--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  6



📊 Student Report
Total Students: 8
Grade 10: 1 student(s)
Grade 9: 2 student(s)
Grade A: 2 student(s)
Grade C: 2 student(s)
Grade B: 1 student(s)

--- Main Menu ---
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  7


👋 Goodbye!


# Summary

**System Capabilities:**

- Secure login system with activity logging
- Add, view, search, update, delete student records
- File-based storage for persistence (`users.txt`, `students.txt`, `activity_log.txt`)
- Generate summary reports of students by grade
- Menu-driven interface for ease of use

**Key Learning Points:**

- File handling (`open`, `read`, `write`, `os.path.exists`)
- User authentication and password input security (`getpass`)
- CRUD operations for structured data
- Logging user actions with timestamps
- Menu-driven console applications in Python

This notebook serves as a **comprehensive reference** for future enhancements or review.
